In [ ]:
#this is a roadmap from beginning truth to the final iteration of iPto19

In [2]:
import cobra

In [7]:
pst_feeder = cobra.io.read_sbml_model("../results/reconstructions/pst_feeder.sbml")
pst_feeder

Name,PST
Memory address,0x07ff740783f28
Number of metabolites,242
Number of reactions,149
Number of groups,0
Objective expression,0
Compartments,"c, e"


WTAF happened between 1 Feeder and v2 with the temp biomass

In [9]:
pst_2 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v2.sbml")
pst_2

Name,PST
Memory address,0x07ff750cdd128
Number of metabolites,1025
Number of reactions,752
Number of groups,0
Objective expression,1.0*PAO1_Biomass - 1.0*PAO1_Biomass_reverse_55fa3
Compartments,"c, e"


In [22]:
pst_3 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v3.sbml")
pst_3.compartments

{'c': '', 'e': '', 'c0': ''}

Between v3 and v4, there is a change in the compartments included in the model. In v3, compartments c, e, and c0 exsist. This creates inconsitancies that are incompatable for cross compartment movement. c and c0 were collapsed into one compartment, c. There is no change in the number of genes or reactions, but there is one less metabolite in v4. 

In [21]:
pst_4 = cobra.io.read_sbml_model("../results/reconstructions/PST_pao1_temp_biomass_v4.sbml")
pst_4.compartments

{'c': '', 'e': ''}